# Sprinkler as a BDI Agent
Belief–Desire–Intention Architecture with Planning and Reconsideration


In [1]:
# ============================================
# SPRINKLER AS A BDI AGENT (ONE-CELL VERSION)
# ============================================

class SprinklerBDIAgent:

    def __init__(self):
        self.target_moisture = 65
        self.beliefs = {}
        self.desires = {"maintain_target": 65, "avoid_waste": True}
        self.intentions = None

    # ------------------------------
    # BELIEF REVISION FUNCTION (BRF)
    # ------------------------------
    def belief_revision(self, profile):
        self.beliefs = {
            "temperature": profile["temp"],
            "humidity": profile["humidity"],
            "soil_moisture": profile["start_moisture"],
            "planned_cycles": profile["planned_cycles"]
        }

    # ------------------------------
    # ENVIRONMENT MODEL
    # ------------------------------
    def evaporation_rate(self):
        return 0.05 * self.beliefs["temperature"] * (1 - self.beliefs["humidity"]/100)

    # ------------------------------
    # OPTION GENERATION + INTENTION
    # ------------------------------
    def generate_intention(self):
        deficit = self.desires["maintain_target"] - self.beliefs["soil_moisture"]

        if deficit > 30:
            minutes = 20
        elif deficit > 20:
            minutes = 15
        elif deficit > 10:
            minutes = 10
        else:
            minutes = 5

        self.intentions = {"action": "water", "duration": minutes}

    # ------------------------------
    # EXECUTION + INTRA-CYCLE RECONSIDERATION
    # ------------------------------
    def execute_intention(self):
        print(f"Adopted Intention: {self.intentions}")

        for minute in range(1, self.intentions["duration"] + 1):
            self.beliefs["soil_moisture"] += 1.5

            if self.beliefs["soil_moisture"] >= self.desires["maintain_target"]:
                print(f"⚡ Reconsideration: Stop early at minute {minute}")
                self.beliefs["soil_moisture"] = self.desires["maintain_target"]
                break

    # ------------------------------
    # ENVIRONMENT TRANSITION
    # ------------------------------
    def environmental_transition(self):
        self.beliefs["soil_moisture"] -= self.evaporation_rate() * 2
        self.beliefs["soil_moisture"] = max(0, self.beliefs["soil_moisture"])

    # ------------------------------
    # GLOBAL RECONSIDERATION
    # ------------------------------
    def emergency_reconsideration(self):
        if self.beliefs["soil_moisture"] < 45:
            print("⚡ Global Reconsideration Triggered")
            emergency_minutes = 0

            while self.beliefs["soil_moisture"] < self.desires["maintain_target"]:
                self.beliefs["soil_moisture"] += 1.5
                emergency_minutes += 1

            print(f"⚡ Emergency watering for {emergency_minutes} minutes")

    # ------------------------------
    # FULL DAY EXECUTION
    # ------------------------------
    def run_day(self, profile, day):
        print(f"\n===== Day {day} =====")
        self.belief_revision(profile)
        print(f"Initial Beliefs: {self.beliefs}")

        for cycle in range(1, self.beliefs["planned_cycles"] + 1):
            print(f"\n--- Cycle {cycle} ---")

            self.generate_intention()
            self.execute_intention()

            print(f"Moisture After Watering: {round(self.beliefs['soil_moisture'],1)}%")

            self.environmental_transition()
            print(f"After Evaporation: {round(self.beliefs['soil_moisture'],1)}%")

            self.emergency_reconsideration()

        print(f"\nEnd of Day {day}: Final Moisture = {round(self.beliefs['soil_moisture'],1)}%")


# ============================================
# SIMULATION DRIVER (5 DAYS)
# ============================================

weather_profiles = {
    "clear": {"temp": 38, "humidity": 35, "start_moisture": 30, "planned_cycles": 3},
    "humid": {"temp": 30, "humidity": 75, "start_moisture": 32, "planned_cycles": 2},
    "hot_humid_extreme": {"temp": 42, "humidity": 40, "start_moisture": 32, "planned_cycles": 2}
}

agent = SprinklerBDIAgent()

for day in range(1, 6):

    while True:
        weather = input(f"Enter weather for Day {day} (clear/humid/hot_humid_extreme): ").lower()
        if weather in weather_profiles:
            break
        else:
            print("Invalid input.")

    agent.run_day(weather_profiles[weather], day)

print("\nSimulation Complete.")

Enter weather for Day 1 (clear/humid/hot_humid_extreme):  humid



===== Day 1 =====
Initial Beliefs: {'temperature': 30, 'humidity': 75, 'soil_moisture': 32, 'planned_cycles': 2}

--- Cycle 1 ---
Adopted Intention: {'action': 'water', 'duration': 20}
Moisture After Watering: 62.0%
After Evaporation: 61.2%

--- Cycle 2 ---
Adopted Intention: {'action': 'water', 'duration': 5}
⚡ Reconsideration: Stop early at minute 3
Moisture After Watering: 65%
After Evaporation: 64.2%

End of Day 1: Final Moisture = 64.2%


Enter weather for Day 2 (clear/humid/hot_humid_extreme):  clear



===== Day 2 =====
Initial Beliefs: {'temperature': 38, 'humidity': 35, 'soil_moisture': 30, 'planned_cycles': 3}

--- Cycle 1 ---
Adopted Intention: {'action': 'water', 'duration': 20}
Moisture After Watering: 60.0%
After Evaporation: 57.5%

--- Cycle 2 ---
Adopted Intention: {'action': 'water', 'duration': 5}
⚡ Reconsideration: Stop early at minute 5
Moisture After Watering: 65%
After Evaporation: 62.5%

--- Cycle 3 ---
Adopted Intention: {'action': 'water', 'duration': 5}
⚡ Reconsideration: Stop early at minute 2
Moisture After Watering: 65%
After Evaporation: 62.5%

End of Day 2: Final Moisture = 62.5%


Enter weather for Day 3 (clear/humid/hot_humid_extreme):  clear


...

Invalid input.


Enter weather for Day 4 (clear/humid/hot_humid_extreme):  humid



===== Day 4 =====
Initial Beliefs: {'temperature': 30, 'humidity': 75, 'soil_moisture': 32, 'planned_cycles': 2}

--- Cycle 1 ---
Adopted Intention: {'action': 'water', 'duration': 20}
Moisture After Watering: 62.0%
After Evaporation: 61.2%

--- Cycle 2 ---
Adopted Intention: {'action': 'water', 'duration': 5}
⚡ Reconsideration: Stop early at minute 3
Moisture After Watering: 65%
After Evaporation: 64.2%

End of Day 4: Final Moisture = 64.2%


Enter weather for Day 5 (clear/humid/hot_humid_extreme):  hot_humid_extreme



===== Day 5 =====
Initial Beliefs: {'temperature': 42, 'humidity': 40, 'soil_moisture': 32, 'planned_cycles': 2}

--- Cycle 1 ---
Adopted Intention: {'action': 'water', 'duration': 20}
Moisture After Watering: 62.0%
After Evaporation: 59.5%

--- Cycle 2 ---
Adopted Intention: {'action': 'water', 'duration': 5}
⚡ Reconsideration: Stop early at minute 4
Moisture After Watering: 65%
After Evaporation: 62.5%

End of Day 5: Final Moisture = 62.5%

Simulation Complete.
